In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/20241114_150555.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/20241114_145527.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_144634.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_155701.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_154546.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_161614.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/20241114_150917.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/20241114_152001.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_155923.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_152354.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_160709.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/20241114_150912.jpg
/kaggle/input/dataset-rice-disease-rm/Tungro/orginal/IMG_20241114_161917.jpg
/kaggle/input/datas

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
import numpy as np
import os

import os
import shutil
import random
from glob import glob
import numpy as np
import tensorflow as tf
import optuna
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

In [23]:
# --- 1. Setup Data Generators (Using the same settings as your notebook) ---
img_size = (224, 224)
batch_size = 32

In [24]:
source_dir = '/kaggle/input/dataset-rice-disease-rm'
train_dir = '/kaggle/working/data/train'
val_dir = '/kaggle/working/data/val'
val_split = 0.2
random.seed(67)
valid_ext = ('.jpg', '.jpeg', '.png')

for label in os.listdir(source_dir):
    class_path = os.path.join(source_dir, label)
    if os.path.isdir(class_path):
        image_paths = []
        for ext in valid_ext:
            image_paths.extend(glob(os.path.join(class_path, '**', f'*{ext}'), recursive=True))
        random.shuffle(image_paths)
        split_idx = int(len(image_paths) * (1 - val_split))
        train_images = image_paths[:split_idx]
        val_images = image_paths[split_idx:]
        os.makedirs(os.path.join(train_dir, label), exist_ok=True)
        os.makedirs(os.path.join(val_dir, label), exist_ok=True)
        for src_path in train_images:
            shutil.copy(src_path, os.path.join(train_dir, label, os.path.basename(src_path)))
        for src_path in val_images:
            shutil.copy(src_path, os.path.join(val_dir, label, os.path.basename(src_path)))

In [25]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Calculate class weights to handle imbalance
class_weights_vals = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights_vals))
num_classes = len(train_generator.class_indices)


Found 8611 images belonging to 5 classes.
Found 2155 images belonging to 5 classes.


In [26]:
# --- 2. Build Model with Best Hyperparameters ---

# Best params = {'dropout_rate': 0.3, 'dense_units': 512, 'optimizer': 'adam', 'learning_rate': 0.0001, 'unfrozen_layers': 40}
BEST_DROPOUT = 0.3
BEST_DENSE_UNITS = 512
BEST_LR = 0.0001
UNFROZEN_LAYERS = 40

In [27]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-UNFROZEN_LAYERS:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(BEST_DENSE_UNITS, activation='relu')(x)
x = Dropout(BEST_DROPOUT)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=BEST_LR)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-6)
]

history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 178s 611ms/step - accuracy: 0.5679 - loss: 0.8912 - val_accuracy: 0.7225 - val_loss: 0.6465 - learning_rate: 1.0000e-04
Epoch 2/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 152s 564ms/step - accuracy: 0.7523 - loss: 0.4908 - val_accuracy: 0.8478 - val_loss: 0.3947 - learning_rate: 1.0000e-04
Epoch 3/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 154s 572ms/step - accuracy: 0.8082 - loss: 0.3707 - val_accuracy: 0.8404 - val_loss: 0.4242 - learning_rate: 1.0000e-04
Epoch 4/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 152s 563ms/step - accuracy: 0.8475 - loss: 0.3229 - val_accuracy: 0.8687 - val_loss: 0.3542 - learning_rate: 1.0000e-04
Epoch 5/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 154s 568ms/step - accuracy: 0.8769 - loss: 0.2671 - val_accuracy: 0.8715 - val_loss: 0.3616 - learning_rate: 1.0000e-04
Epoch 6/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 154s 571ms/step - accuracy: 0.8886 - loss: 0.2382 - val_accuracy: 0.8729 - val_loss: 0.3457 - learning_rate: 1.0000e-04
Epoch 7/100
270/270 ━━━━━━━━━━━━━━

In [29]:
save_path = "best_resnet50_final.h5"
model.save(save_path)
print(f"Model saved successfully to {save_path}")


from IPython.display import FileLink
display(FileLink(save_path))

Model saved successfully to best_resnet50_final.h5


/kaggle/working/best_resnet50_final.h5